In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Clone and install rabtize
!git clone -q https://git.sr.ht/~rehandaphedar/rabtize
!sed -i 's/>=3\.13/>=3.10/' rabtize/pyproject.toml
!pip install -q -e rabtize/.[embed]

# Clone lafzize
!git clone -q https://git.sr.ht/~rehandaphedar/lafzize
!pip install -q -r lafzize/requirements.txt

# Install Go and jumlize
!wget -q https://go.dev/dl/go1.25.5.linux-amd64.tar.gz
!tar -C /usr/local -xzf go1.25.5.linux-amd64.tar.gz
!/usr/local/go/bin/go install git.sr.ht/~rehandaphedar/jumlize/v3@latest

# Install quran_segmenter
!git clone https://github.com/ashaltu/quran_segmenter.git
!cd quran_segmenter/ && pip install -q -e .

In [21]:
# Set needed environment variables. Make sure these files are uploaded.
import os

env_vars = {
    "LAFZIZE_WORDS": "/content/qpc-hafs-word-by-word.json",
    "LAFZIZE_METADATA": "/content/quran-metadata-misc.json",
    "QURAN_SEGMENTER_CONFIG": "/content/quran_data/config.json"
}

for k, v in env_vars.items():
    os.environ[k] = v

env_vars

!cp quran-metadata-misc.json lafzize/.
!cp qpc-hafs-word-by-word.json lafzize/.
!cp qpc-hafs-word-by-word.json rabtize/.

In [3]:
# Init the quran data folder where embeddings and translations are placed
!quran-segmenter init --data-dir ./quran_data

# (Optional) If you already have embeddings, spans.npz and/or the language
# specific .npz embeddings file, copy them over from your Drive.
# !cp -rf /content/drive/MyDrive/QuranSegmenter/embeddings/* ./quran_data/embeddings/.

# (Optional) Download spans.npz and english embeddings
# !cd /content/quran_data/embeddings && git clone -q https://huggingface.co/datasets/rehandaphedar/rabtize .

Initialized configuration at quran_data/config.json
Data directory: quran_data


In [6]:
!quran-segmenter register ghali-apapur-apaghuna-clean /content/ghali-apapur-apaghuna-clean.json \
--name "Oromo Translation" \
--language om \
--spans-embeddings-filepath /content/quran_data/embeddings/spans.npz \
--segment-embeddings-filepath /content/quran_data/embeddings/ghali-apapur-apaghuna-clean.npz

2025-12-08 19:58:37 [INFO] quran_segmenter.config: Loading existing config from /content/quran_data/config.json
2025-12-08 19:58:37 [INFO] quran_segmenter.config: Config reloaded from /content/quran_data/config.json
2025-12-08 19:58:37 [INFO] quran_segmenter.config: Copied translation to /content/quran_data/translations/ghali-apapur-apaghuna-clean.json
2025-12-08 19:58:37 [INFO] quran_segmenter.config: Translation ghali-apapur-apaghuna-clean appears to be pre-segmented
2025-12-08 19:58:37 [INFO] quran_segmenter.config: Registered translation: ghali-apapur-apaghuna-clean
2025-12-08 19:58:37 [INFO] quran_segmenter.pipeline.orchestrator: Registered translation: ghali-apapur-apaghuna-clean
Registered translation: ghali-apapur-apaghuna-clean


In [7]:
!quran-segmenter prepare ghali-apapur-apaghuna-clean #--skip-segmentation --skip-embeddings

2025-12-08 19:58:43 [INFO] quran_segmenter.config: Loading existing config from /content/quran_data/config.json
2025-12-08 19:58:43 [INFO] quran_segmenter.config: Config reloaded from /content/quran_data/config.json
2025-12-08 19:58:43 [INFO] quran_segmenter.pipeline.orchestrator: Segmentation already done for ghali-apapur-apaghuna-clean
2025-12-08 19:58:43 [INFO] quran_segmenter.pipeline.orchestrator: Spans embeddings already exist
2025-12-08 19:58:43 [INFO] quran_segmenter.pipeline.orchestrator: Segment embeddings already exist for ghali-apapur-apaghuna-clean
2025-12-08 19:58:43 [INFO] quran_segmenter.pipeline.orchestrator: Translation ghali-apapur-apaghuna-clean is ready for processing

Preparation Status:
  segmentation: already_done
  spans_embeddings: already_done
  segment_embeddings: already_done

✓ Translation 'ghali-apapur-apaghuna-clean' is ready for processing


In [27]:
# @markdown **Verse Specification:**
# @markdown - Single verse: `"2:282"`
# @markdown - Verse range: `"2:255-260"`
# @markdown - Entire surah: `"2"` (all 286 verses)

# @markdown **Note**: The .mp3 file must contain audio of the verses specified
!quran-segmenter process \
    19.mp3 \
    "19" \
    ghali-apapur-apaghuna-clean \
    -o output.json \
    --start-server

2025-12-08 20:20:51 [INFO] quran_segmenter.config: Loading existing config from /content/quran_data/config.json
2025-12-08 20:20:51 [INFO] quran_segmenter.config: Config reloaded from /content/quran_data/config.json
2025-12-08 20:20:51 [INFO] quran_segmenter.pipeline.orchestrator: Processing 19.mp3 for verses 19:1-98
2025-12-08 20:20:51 [INFO] quran_segmenter.pipeline.orchestrator: Step 1/3: Getting word timestamps (lafzize)
2025-12-08 20:20:52 [INFO] quran_segmenter.utils.server: Starting lafzize server on port 8004...
2025-12-08 20:20:52 [INFO] quran_segmenter.utils.server: Waiting for lafzize server to be ready...
2025-12-08 20:21:48 [INFO] quran_segmenter.utils.server: ✓ Lafzize server is ready
2025-12-08 20:21:48 [INFO] quran_segmenter.pipeline.lafzize: Calling lafzize for 19.mp3 verses 19:1-98
2025-12-08 20:22:00 [INFO] quran_segmenter.pipeline.lafzize: Got 961 word timestamps
2025-12-08 20:22:00 [INFO] quran_segmenter.pipeline.orchestrator: Step 2/3: Getting alignment (rabtize)
